In [6]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)


tensor([[ 0.2795,  0.0397, -0.1300,  0.1092,  0.0539,  0.1057, -0.0455, -0.2469,
         -0.0220,  0.4667],
        [ 0.0539,  0.1269, -0.2124,  0.1060, -0.1609,  0.0746,  0.1337, -0.2479,
          0.0450,  0.1915]], grad_fn=<AddmmBackward0>)

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [8]:
net = MLP()
net(X)

tensor([[ 0.1350, -0.0951,  0.1217, -0.2583,  0.1859,  0.0563, -0.2161,  0.1149,
         -0.0394,  0.2134],
        [ 0.2297,  0.0276,  0.1771, -0.1426,  0.2614,  0.1259, -0.2231,  0.0704,
         -0.1290,  0.0571]], grad_fn=<AddmmBackward0>)

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[-0.0038, -0.1057,  0.0965,  0.1219,  0.0895,  0.0713, -0.1751, -0.2589,
         -0.0323, -0.0292],
        [-0.1117, -0.0658,  0.1979, -0.0300,  0.0296, -0.1015,  0.0268, -0.2360,
          0.0438, -0.0006]], grad_fn=<AddmmBackward0>)

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)


tensor(0.0606, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(0.0573, grad_fn=<SumBackward0>)

# 参数管理

In [13]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

X = torch.rand(size=(2, 4))
net(X)


tensor([[0.2353],
        [0.4000]], grad_fn=<AddmmBackward0>)

In [14]:
print(net[2].state_dict())


OrderedDict([('weight', tensor([[ 0.0528, -0.0688, -0.1777, -0.0821,  0.2394,  0.3255, -0.2914,  0.3417]])), ('bias', tensor([0.3069]))])


In [15]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3069], requires_grad=True)
tensor([0.3069])


In [16]:
net[2].weight.grad == None

True

In [17]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])



('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:

net.state_dict()['2.bias'].data

tensor([0.3069])

In [19]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[0.1123],
        [0.1123]], grad_fn=<AddmmBackward0>)

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [21]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

# 对某些块使用不同的初始化方法

In [22]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([-0.4074, -0.6799, -0.2836, -0.5541])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [23]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 5.5965,  0.0000,  6.2737,  7.1259],
        [-9.5059,  6.0737,  8.2594, -0.0000]], grad_fn=<SliceBackward0>)

In [24]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.6185,  0.2127, -0.2820,  0.1093])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -5.2019,  0.0000, -0.0000],
        [-9.0081, -6.2525, -0.0000, -6.7259]], grad_fn=<SliceBackward0>)

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]


tensor([42.0000, -4.2019,  1.0000,  1.0000])

# 参数绑定

In [27]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [28]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2


tensor([0, 1, 2, 3])

In [29]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')

x2, y2 = torch.load('x-files')
(x2, y2)

mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')

mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [30]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)


In [31]:
torch.save(net.state_dict(), 'mlp.params')


In [ ]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

Y_clone = clone(X)
Y_clone == Y
